In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00


In [ ]:
pip install requests

In [ ]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=9bac26ca394543dc004b44cf8dc607bb9be37802c798ab1bdc42f0d5ebed826c
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import openai

openai.api_key = ""

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrap_google_news(keyword: str, limit=5):
    #https://www.google.com/search?q=%ED%94%84%EB%A1%AC%ED%94%84%ED%8A%B8%20%EC%97%94%EC%A7%80%EB%8B%88%EC%96%B4%EB%A7%81&tbm=nws&num=5
    google_search_url = 'https://www.google.com/search'

    params = {'q': keyword, 'tbm': 'nws', 'num': limit}

    headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
    }

    res = requests.get(google_search_url, params=params, headers=headers)

    soup = BeautifulSoup(res.content, 'html.parser')

    news_results = []
    for el in soup.select("div.SoaBEf"):
        news_results.append(
            {
                "link": el.find("a")["href"],
                "title": el.select_one("div.MBeuO").get_text(),
                "snippet": el.select_one(".GI74Re").get_text()
            }
        )

    return news_results

In [ ]:
import json

results = scrap_google_news('프롬프트 엔지니어링')

print(json.dumps(results, ensure_ascii=False, indent=2))

[
  {
    "link": "https://www.ciokorea.com/news/306751",
    "title": "인터뷰 | “시멘틱 데이터로 프롬프트 엔지니어링 패러다임 전환” 키토크AI 도준웅 대표",
    "snippet": "생성형 AI 기술 경쟁이 본격화되는 모양새다. 대기업, 스타트업 가릴 것 없이 각자의 철학을 담은 생성형 AI 기술이 더 쏟아져 나오고 \n있다."
  },
  {
    "link": "https://www.asiae.co.kr/article/2023091208450030787",
    "title": "\"AI 다룰 줄 아세요?\" 채용공고 250% 폭증…美서 '귀한 몸'됐다",
    "snippet": "생성형 AI 채용 공고 2년간 250% 증가 자사 인재 부족…프리랜서 전문가 찾아 '프롬프트 엔지니어링' 등 AI 관련 검색↑. 미국 \n프리랜서 고용 시장에서..."
  },
  {
    "link": "https://zdnet.co.kr/view/?no=20230316150325",
    "title": "프롬프트 엔지니어링은 직업이라기보다 역량이다",
    "snippet": "GPT-4 같은 초거대AI가 등장하면서 이른바 '프롬프트 엔지니어'라는 새로운 직군이 생겼다고 한다. 프롬프트는 초거대AI와 소통하는 \n창(窓)이자 소통..."
  },
  {
    "link": "https://www.aitimes.com/news/articleView.html?idxno=152929",
    "title": "\"프롬프트 엔지니어링으로 GPT-4 수능 국어 상위 5% 도달\"",
    "snippet": "프로젝트 팀 노마다마스는 프롬프트 엔지니어링으로 'GPT-4' 성능을 강화, 수능 국어 2등급(상위 5%)을 달성했다고 17일 밝혔다."
  },
  {
    "link": "https://www.itworld.co.kr/news/283801",
    "title": "“AI, 입맛에 맞게 

In [ ]:
def news_gpt(messages, temperature=0, max_tokens=1024):
    functions = [
        {
            "name": "scrap_google_news",
            "description": "구글에서 뉴스를 검색합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "구글에 검색할 키워드",
                    }
                },
                "required": ["keyword"],
            },
        }
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return completion.choices[0].message

In [ ]:
result = news_gpt([
    {
        "role": "user",
        "content": "세계 경제에 대한 뉴스를 요약해줘"
    }
])

result

<OpenAIObject at 0x7d7a4c34a340> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "scrap_google_news",
    "arguments": "{\n  \"keyword\": \"\uc138\uacc4 \uacbd\uc81c\"\n}"
  }
}

In [ ]:
import json

arguments = json.loads(result.function_call.arguments)

arguments

{'keyword': '세계 경제'}

In [ ]:
results = scrap_google_news(arguments['keyword'])

print(json.dumps(results, ensure_ascii=False, indent=2))

[
  {
    "link": "https://www.sedaily.com/NewsView/29UNLPWZ6Y",
    "title": "푸틴 '美·유럽이 세계 경제 시스템 파괴'",
    "snippet": "블라디미르 푸틴 러시아 대통령이 12일 블라디보스토크에서 열리고 있는 동방경제포럼(EEF) 전체회의 연설에서 \"유럽과 미국은 스스로 \n구축해 온 세계..."
  },
  {
    "link": "https://m.khan.co.kr/article/202309121637011",
    "title": "급증한 글로벌 기업 부채, 금융위기 때보다 2배로 치솟아···세계 경제 뇌관되나",
    "snippet": "세계 주요국 중앙은행의 금리가 상승하면서 글로벌 기업들의 부채 부담도 치솟고 있다. 전문가들..."
  },
  {
    "link": "https://www.mk.co.kr/news/economy/10828064",
    "title": "美 금리 인상 막바지 … 하이일드채권에서도 투자 기회",
    "snippet": "CEO 라운드테이블 : 2024년 기회와 위협존슨 프랭클린템플턴 회장\"고물가 수준은 당분간 지속\"내년 세계경제 최대변수론지정학적 \n역학관계 꼽아."
  },
  {
    "link": "https://www.hani.co.kr/arti/economy/economy_general/1087504.html",
    "title": "미국 침체 신호, 중국 굼뜬 회복…세계경제 불확실성 더 커져",
    "snippet": "“최근 금융시장 변동성이 증가하고 여러 지표가 서로 다른 방향을 가리키면서 세계경제에 안개가 짙어졌다. 올해 세계경제는 험난한 회복 \n과정(A..."
  },
  {
    "link": "https://www.edaily.co.kr/news/read?newsId=01380886635738808&mediaCodeNo=257",
    "title": "현대경제硏 \"세계경제 피크아웃·

In [ ]:
result = news_gpt([
    {
        "role": "user",
        "content": json.dumps(results, ensure_ascii=False)
    },
    {
        "role": "user",
        "content": "세계 경제에 대한 뉴스를 요약해줘"
    }
])

result

<OpenAIObject at 0x7d7a4e86b150> JSON: {
  "role": "assistant",
  "content": "1. \"\ud478\ud2f4 '\u7f8e\u00b7\uc720\ub7fd\uc774 \uc138\uacc4 \uacbd\uc81c \uc2dc\uc2a4\ud15c \ud30c\uad34'\" - \ub7ec\uc2dc\uc544 \ub300\ud1b5\ub839 \ud478\ud2f4\uc740 \ub3d9\ubc29\uacbd\uc81c\ud3ec\ub7fc\uc5d0\uc11c \uc720\ub7fd\uacfc \ubbf8\uad6d\uc774 \uc138\uacc4 \uacbd\uc81c \uc2dc\uc2a4\ud15c\uc744 \ud30c\uad34\ud558\uace0 \uc788\ub2e4\uace0 \ube44\ud310\ud588\uc2b5\ub2c8\ub2e4. (\ucd9c\ucc98: \uc138\uacc4\uc77c\ubcf4)\n\n2. \"\uae09\uc99d\ud55c \uae00\ub85c\ubc8c \uae30\uc5c5 \ubd80\ucc44, \uae08\uc735\uc704\uae30 \ub54c\ubcf4\ub2e4 2\ubc30\ub85c \uce58\uc19f\uc544\u00b7\u00b7\u00b7\uc138\uacc4 \uacbd\uc81c \ub1cc\uad00\ub418\ub098\" - \uc138\uacc4 \uc8fc\uc694\uad6d \uc911\uc559\uc740\ud589\uc758 \uae08\ub9ac \uc0c1\uc2b9\uc73c\ub85c \uc778\ud574 \uae00\ub85c\ubc8c \uae30\uc5c5\ub4e4\uc758 \ubd80\ucc44 \ubd80\ub2f4\uc774 \ud06c\uac8c \uc99d\uac00\ud558\uace0 \uc788\uc2b5\ub2c8\ub2e4. \uc774\ub294 \u

In [ ]:
print(result['content'])

1. "푸틴 '美·유럽이 세계 경제 시스템 파괴'" - 러시아 대통령 푸틴은 동방경제포럼에서 유럽과 미국이 세계 경제 시스템을 파괴하고 있다고 비판했습니다. (출처: 세계일보)

2. "급증한 글로벌 기업 부채, 금융위기 때보다 2배로 치솟아···세계 경제 뇌관되나" - 세계 주요국 중앙은행의 금리 상승으로 인해 글로벌 기업들의 부채 부담이 크게 증가하고 있습니다. 이는 세계 경제에 대한 불확실성을 증가시킬 수 있습니다. (출처: 경향신문)

3. "美 금리 인상 막바지 … 하이일드채권에서도 투자 기회" - 미국의 금리 인상이 막바지에 접어들면서 하이일드채권에서도 투자 기회가 생길 수 있습니다. 이는 투자자들에게 긍정적인 영향을 줄 수 있습니다. (출처: 매일경제)

4. "미국 침체 신호, 중국 굼뜬 회복…세계경제 불확실성 더 커져" - 최근 금융시장 변동성이 증가하고 여러 지표가 서로 다른 방향을 가리키면서 세계경제에 대한 불확실성이 커지고 있습니다. 미국의 침체 신호와 중국의 회복 신호가 혼재하고 있습니다. (출처: 한겨레)

5. "현대경제硏 '세계경제 피크아웃·달러 독주 등…리스크 최소화해야'" - 현재 진행 중인 글로벌 리스크인 세계경제 피크아웃과 미 달러화 독주 등은 올해와 내년에도 국내경제에 영향을 미칠 수 있는 요인입니다. 이에 대한 리스크 최소화가 필요합니다. (출처: 이데일리)


In [ ]:
result = news_gpt([
    {
        "role": "user",
        "content": json.dumps(results, ensure_ascii=False)
    },
    {
        "role": "user",
        "content": "세계 경제에 대한 뉴스를 요약해줘. 뉴스를 종합하여 제목과 본문이 있는 새로운 블로그 글로 작성해줘."
    }
])

result

<OpenAIObject at 0x7d7a32051850> JSON: {
  "role": "assistant",
  "content": "\uc138\uacc4 \uacbd\uc81c\uc5d0 \ub300\ud55c \ub274\uc2a4\ub97c \uc885\ud569\ud558\uc5ec \uc0c8\ub85c\uc6b4 \ube14\ub85c\uadf8 \uae00\uc744 \uc791\uc131\ud574\ubcf4\uaca0\uc2b5\ub2c8\ub2e4.\n\n\uc81c\ubaa9: \uc138\uacc4 \uacbd\uc81c\uc758 \ubd88\ud655\uc2e4\uc131\uacfc \ub9ac\uc2a4\ud06c, \ud604\ud669\uacfc \uc804\ub9dd\n\n\ubcf8\ubb38:\n\uc138\uacc4 \uacbd\uc81c\ub294 \ud604\uc7ac \ubd88\ud655\uc2e4\uc131\uacfc \ub9ac\uc2a4\ud06c\uc5d0 \uc9c1\uba74\ud558\uace0 \uc788\uc2b5\ub2c8\ub2e4. \ub7ec\uc2dc\uc544 \ub300\ud1b5\ub839\uc778 \ube14\ub77c\ub514\ubbf8\ub974 \ud478\ud2f4\uc740 \ub3d9\ubc29\uacbd\uc81c\ud3ec\ub7fc\uc5d0\uc11c \uc720\ub7fd\uacfc \ubbf8\uad6d\uc774 \uc138\uacc4 \uacbd\uc81c \uc2dc\uc2a4\ud15c\uc744 \ud30c\uad34\ud558\uace0 \uc788\ub2e4\uace0 \uc8fc\uc7a5\ud558\uc600\uc2b5\ub2c8\ub2e4. \uc774\ub7ec\ud55c \uc8fc\uc7a5\uc740 \uc138\uacc4 \uacbd\uc81c\uc758 \uc548\uc815\uc131\uc5d0 \ub300\ud55c \u

In [ ]:
print(result['content'])